# Demo 6 - Recurrent Neural Networks (RNNs)

### Cybersecurity MBA - Inteligência Artificial e Machine Learning (SEG)

### Prof. Dr. Ahirton Lopes (profahirton.lopes@fiap.com.br)

### Recurrent Neural Networks usando Keras e a base SMS Spam Collection Data (mensagens spam rotuladas)

### Prof. Dr. José Ahirton Batista Lopes Filho

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

from keras.layers import SimpleRNN, LSTM, GRU, Embedding, Dense, Flatten
from keras.models import Sequential

import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model

In [2]:
def message_to_array(msg):
    msg = msg.lower().split(' ')
    test_seq = np.array([word_index[word] for word in msg])

    test_seq = np.pad(test_seq, (500-len(test_seq), 0), 'constant', constant_values=(0))
    test_seq = test_seq.reshape(1, 500)
    return test_seq

### Carregando a base SMS Spam Collection Data (sms rotulados em spam e ham)

*** Mais informações em https://www.kaggle.com/uciml/sms-spam-collection-dataset e https://archive.ics.uci.edu/ml/datasets/sms+spam+collection ***

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Arquivo de nome "{name}" de tamanho {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving spam_data.csv to spam_data.csv
Arquivo de nome "spam_data.csv" de tamanho 485703 bytes


In [4]:
# Importando nossos dados a partir do arquivo .csv

data = pd.read_csv('spam_data.csv', sep=',')

## Pré processamento de mensagens e rótulos

In [5]:
messages = []
labels = []

for index, row in data.iterrows():
    messages.append(row['Message'])
    if row['Category'] == 'ham': # classificando em 0 e 1
        labels.append(0)
    else:
        labels.append(1)

In [6]:
text_data_train = np.asarray(messages)
text_data_target = np.asarray(labels)

print("Numero de mensagens: ", len(messages))
print("Numero de rotulos: ", len(labels))

Numero de mensagens:  5572
Numero de rotulos:  5572


## Verificando o formato de nossas mensagens e rótulos

In [7]:
text_data_train[:5]

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       'U dun say so early hor... U c already then say...',
       "Nah I don't think he goes to usf, he lives around here though"],
      dtype='<U910')

In [8]:
text_data_target[:5]

array([0, 0, 1, 0, 0])

## Processamento de nosso vocabulário

In [9]:
max_vocab = 10000 # Ignorando todas as palavras, exceto as 10.000 palavras mais comuns em nosso vocabulário
max_len = 500 # max_len especifica o comprimento máximo da sequência (que é truncada se for longa, e preenchida - via padding - se mais curta)

tokenizer = Tokenizer(num_words=max_vocab)

# Calculando a frequência das palavras em nossos dados de treinamento

tokenizer.fit_on_texts(text_data_train)

# Convertendo um array de mensagens para uma lista de sequências de inteiros

sequences = tokenizer.texts_to_sequences(text_data_train)

# Criação de dicionário afim de registrar as palavras para um índice do tipo inteiro

word_index = tokenizer.word_index

# Convertendo a matriz de sequências de inteiros em uma matriz 2D (com preenchimento, quando necessário)

data = pad_sequences(sequences, maxlen=max_len)

print("data shape: ", data.shape)

data shape:  (5572, 500)


## Dividindo o dataset em treinamento e validação

In [10]:
# Usando 80% dos dados disponíveis para treinamento e validação (80% treinamento, 20% validação), bem como 20% para teste

train_samples = int(len(text_data_train)*0.8)

messages_train = data[:train_samples]
labels_train = text_data_target[:train_samples]

messages_test = data[train_samples:len(text_data_train)-2]
labels_test = text_data_target[train_samples:len(text_data_train)-2]

embedding_mat_columns=32

## Definindo a arquitetura do modelo (IMPORTANTE!)

* 3 camadas (Camadas de Embedding, Camada RNN Simples e Densa);
* Uma última camada do tipo totalmente conectada contendo 1 neurônio.

In [11]:
# Construindo nosso modelo de RNN Simples (via camada SimpleRNN)

model = Sequential()

# Utilização de camada de Embedding tendo em vista converter a codificação de inteiros em embeddings de palavras
# O modelo vai então aprender a partir da matriz de embedding durante o treinamento;

model.add(Embedding(input_dim=max_vocab, output_dim=embedding_mat_columns, input_length=max_len)) # A matriz de embedding contém max_vocab como número de linhas e o número de colunas escolhido

model.add(SimpleRNN(units=embedding_mat_columns))

# Após o descrito na Aula 6, testem com novas arquiteturas de camadas (RNNs, LSTMs e GRUs Bidirecionais e stackings) (!)

#model.add(LSTM(units=embedding_mat_columns))
#model.add(GRU(units=embedding_mat_columns))

model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## Compilando o modelo

In [12]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Treinamento de nosso modelo

In [13]:
model.fit(messages_train, labels_train, epochs=100, batch_size=16, validation_split=0.2)

Epoch 1/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 23s 66ms/step - acc: 0.8879 - loss: 0.3222 - val_acc: 0.9608 - val_loss: 0.1066
Epoch 2/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - acc: 0.9762 - loss: 0.0783 - val_acc: 0.9843 - val_loss: 0.0516
Epoch 3/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - acc: 0.9888 - loss: 0.0425 - val_acc: 0.9821 - val_loss: 0.0576
Epoch 4/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - acc: 0.9942 - loss: 0.0258 - val_acc: 0.9809 - val_loss: 0.0686
Epoch 5/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - acc: 0.9921 - loss: 0.0287 - val_acc: 0.9821 - val_loss: 0.0612
Epoch 6/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - acc: 0.9954 - loss: 0.0181 - val_acc: 0.9832 - val_loss: 0.0652
Epoch 7/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - acc: 0.9969 - loss: 0.0109 - val_acc: 0.9821 - val_loss: 0.0644
Epoch 8/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - acc: 0.9972 - loss: 0.0084 - val_acc: 0.9765 - val_loss: 0.0784
Epoch 9/100
223/223 ━━━━━━━━━━━━━━

## Teste de nosso modelo

In [14]:
prediction = model.predict(messages_test)
accuracy = model.evaluate(messages_test, labels_test)
print("Perda em teste {0:.2f} acuracia eh {1:.2f}  ".format(accuracy[0],accuracy[1]))

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - acc: 0.9734 - loss: 0.1574
Perda em teste 0.15 acuracia eh 0.98  


In [15]:
# Construindo mensagem personalizada para teste rápido

#custom_msg = 'Hi how are you'
custom_msg = 'Free entry in 2 a wkly comp to win FA Cup final tkts 21st May Text FA to 87121 to receive entry question'

test_seq = message_to_array(custom_msg)
prediction = (model.predict(test_seq) > 0.5).astype("int32")
print(prediction)  #[1] para spam [0] para nao spam (ham)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
[[1]]
